In [ ]:
%matplotlib inline
import matplotlib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from SimpleDeepESN import *
from IPython.display import display, clear_output

train_data= 0.5*(2*np.random.rand(6000)-1)
test_data= 0.5*(2*np.random.rand(2000)-1)

wash_seq = train_data[:1000]
train_x = train_data[1000:5000]
train_y = np.vstack( list( train_data[1000-k:5000-k] for k in range(100) ) )

In [ ]:
def inc_hebb_train(esn,train_data,test_data,max_epochs,step,train_fun):
  for i in range( esn.N ):
    esn.train(train_x,train_y,wash_seq,i) 
    init_mc = MCi(esn,i,test_data)
    max_mc = init_mc ;  max_win=np.copy(esn.ress[i].W_in); max_w=np.copy(esn.ress[i].W);

    for epoch in range(max_epochs):
      train_fun(esn,i,train_data,step)

      esn.train(train_x,train_y,wash_seq,i)
      act_mc = MCi(esn,i,test_data)

      if max_mc < act_mc : 
        max_mc = act_mc; max_win=np.copy(esn.ress[i].W_in); max_w=np.copy(esn.ress[i].W);
      
    esn.ress[i].W_in = max_win;  esn.ress[i].W = max_w;


In [ ]:
init_mcs=[None]*5
final_mcs=[None]*5

for g in range(5):
  esn = DeepESN( N=5, Nr=50, Ny=100 , rho=0.1 , r_density=0.8 , i_density=1 )
  
  for i in range(esn.N):
    esn.train(train_x,train_y,wash_seq,i)
  init_mcs[g] = MC(esn,test_data)
  
  inc_hebb_train(esn, train_data, test_data, 300, step, inc_train_both)
  
  for i in range(esn.N):
    esn.train(train_x,train_y,wash_seq,i)
  final_mcs[g] = MC(esn,test_data)

init_mcs=np.array(init_mcs)
final_mcs=np.array(final_mcs)

init_means = [None]*5
final_means = [None]*5
ipms = [None]*5

for i in range(5):
  init_means[i] = np.mean(init_mcs[:,i])
  final_means[i] = np.mean(final_mcs[:,i])
  ipms[i] = np.mean( np.divide( abs(init_mcs[:,i]-final_mcs[:,i]), init_mcs[:,i])*100 )

print(init_means)
print(final_means)
print(ipms)


[8.453638616157631, 9.013333685584186, 9.400610749572792, 9.680383064933142, 9.917825053677877]
[17.995333460080474, 22.730257672196764, 23.257883185697484, 24.851925302347667, 20.265570502256583]
[112.81630272193588, 152.14956234589286, 147.39717186151648, 156.82710577823792, 140.14777054277798]


In [ ]:
init_mcs=[None]*5
final_mcs=[None]*5

for g in range(5):
  print(g)
  esn = DeepESN( N=5, Nr=50, Ny=100 , rho=0.9 , r_density=0.8 , i_density=1 )
  
  for i in range(esn.N):
    esn.train(train_x,train_y,wash_seq,i)
  init_mcs[g] = MC(esn,test_data)
  
  inc_hebb_train(esn, train_data, test_data, 80, step, inc_train_both)
  
  for i in range(esn.N):
    esn.train(train_x,train_y,wash_seq,i)
  final_mcs[g] = MC(esn,test_data)

init_mcs=np.array(init_mcs)
final_mcs=np.array(final_mcs)

init_means = [None]*5
final_means = [None]*5
ipms = [None]*5

for i in range(5):
  init_means[i] = np.mean(init_mcs[:,i])
  final_means[i] = np.mean(final_mcs[:,i])
  ipms[i] = np.mean( np.divide( abs(init_mcs[:,i]-final_mcs[:,i]), init_mcs[:,i])*100 )

print(init_means)
print(final_means)
print(ipms)


0
1
2
3
4
[19.101524111814207, 25.73946695810822, 30.87704331149372, 35.672366582432474, 38.945519822094674]
[18.11894284433478, 26.113304233331377, 42.374247464709285, 42.034105538779514, 45.65719271571093]
[12.371206595284622, 21.898697536147903, 59.93715087718501, 40.73550455736158, 40.349013929761824]


In [ ]:
################# allenamento globale DeepESN ##################
step= [1e-05,-1e-05]
max_epochs= 250

dims=[None]*10

for i in range(10):
  clear_output()
  print(i, dims[i-1])
  esn = DeepESN( N=5, Nr=50, Ny=100 , rho=0.1 , r_density=0.8 , i_density=1 )

  esn.glob_train(train_x,train_y,wash_seq)
  init_mc = glob_MC(esn,test_data)
  max_mc = init_mc ; max_ind=0

  for epoch in range(max_epochs):
    glob_train_both(esn,train_data,step)

    esn.glob_train(train_x,train_y,wash_seq)
    act_mc = glob_MC(esn,test_data)

    if max_mc < act_mc : 
      max_mc = act_mc; max_ind=epoch; max_dss = glob_DSS(esn,test_data)
  
  dims[i] = [ init_mc , max_mc , max_ind , max_dss , epoch ]

a=np.array(dims).reshape(10,5)

dims = pd.DataFrame( a )
dims.columns=['dim. prima','max dim', 'epoch' ,'dim. dopo','epoche eseguite']
print("ANTI-HEBBIANO REC , i_density=0.2 ")
print(dims.to_markdown()) 

print('media e varianza dimensionne iniziale:', np.mean(a[:,0]),np.var(a[:,0]) )
print('media e varianza massimi dimensione: ',np.mean(a[:,1]),np.var(a[:,1]) )
print('media e varianza epoche per massima dimensione: ',np.mean(a[:,2]),np.var(a[:,2]))
print('media e varianza massima dimensione: ',np.mean(a[:,3]),np.var(a[:,3]))
print('media e varianza incremento dimensione:', np.mean( np.divide( abs(a[:,0]-a[:,1]), a[:,0])*100 ),'%' , np.var( np.divide( abs(a[:,0]-a[:,1]), a[:,0])*100 ),'%'  )



9 [17.232914898246086, 54.411062033766, 204, 1.0002085872267399, 249]
ANTI-HEBBIANO REC , i_density=0.2 
|    |   dim. prima |   max dim |   epoch |   dim. dopo |   epoche eseguite |
|---:|-------------:|----------:|--------:|------------:|------------------:|
|  0 |      17.2469 |   51.6823 |     201 |     1.0011  |               249 |
|  1 |      16.7367 |   55.9257 |     201 |     1.0003  |               249 |
|  2 |      17.0597 |   69.5399 |     206 |     1.00014 |               249 |
|  3 |      17.2034 |   48.9975 |     194 |     1.00023 |               249 |
|  4 |      17.0804 |   54.2459 |     202 |     1.00011 |               249 |
|  5 |      17.1045 |   56.8555 |     203 |     1.00127 |               249 |
|  6 |      16.8992 |   45.4124 |     199 |     1.00047 |               249 |
|  7 |      16.986  |   56.1767 |     205 |     1.00019 |               249 |
|  8 |      17.2329 |   54.4111 |     204 |     1.00021 |               249 |
|  9 |      17.2607 |   46.4418 |    

In [ ]:

init_mcs=[None]*5
final_mcs=[None]*5

init_dss=[None]*5
final_dss=[None]*5

for g in range(5):
  esn = DeepESN( N=5, Nr=50, Ny=100 , rho=0.9 , r_density=0.2 , i_density=0.1 )
  
  for i in range(esn.N):
    esn.train(train_x,train_y,wash_seq,i)
  init_mcs[g] = MC(esn,test_data)
  init_dss[g] = DSS(esn,test_data)
  
  for epoch in range(1000):
    inc_train_input(esn,0,train_data,-1e-05)

  for i in range( 1,esn.N ):
    best_Win = np.copy(esn.ress[i].W_in)
    best_DSS = DSSi(esn,test_data,i)
    
    print('res:', i )
    for epch in range(500):
      inc_train_input(esn,i,train_data,-1e-05)
      
      act_DSS = DSSi(esn,test_data,i)
      if best_DSS < act_DSS:
        best_DSS = act_DSS
        best_Win = np.copy(esn.ress[i].W_in)

    esn.ress[i].W_in = best_Win
  
  for i in range(esn.N):
    esn.train(train_x,train_y,wash_seq,i)
  final_mcs[g] = MC(esn,test_data)
  final_dss[g] = DSS(esn,test_data)

init_mcs=np.array(init_mcs)
final_mcs=np.array(final_mcs)

init_dss=np.array(init_dss)
final_dss=np.array(final_dss)

init_means = [None]*5
final_means = [None]*5
ipms = [None]*5

for i in range(5):
  init_means[i] = np.mean(init_mcs[:,i])
  final_means[i] = np.mean(final_mcs[:,i])
  ipms[i] = np.mean( np.divide( abs(init_mcs[:,i]-final_mcs[:,i]), init_mcs[:,i])*100 )

print('MC')
print(init_means)
print(final_means)
print(ipms)

init_means = [None]*5
final_means = [None]*5
ipms = [None]*5

for i in range(5):
  init_means[i] = np.mean(init_dss[:,i])
  final_means[i] = np.mean(final_dss[:,i])
  ipms[i] = np.mean( np.divide( abs(init_dss[:,i]-final_dss[:,i]), init_dss[:,i])*100 )

print('DSS')
print(init_means)
print(final_means)
print(ipms)




res: 1
res: 2
res: 3
res: 4
res: 1
res: 2
res: 3
res: 4
res: 1
res: 2
res: 3
res: 4
res: 1
res: 2
res: 3
res: 4
res: 1
res: 2
res: 3
res: 4
MC
[20.178483057737147, 25.875156548688334, 30.987931683920813, 35.22135939051396, 39.330042023876324]
[38.107042591917754, 48.00024911413115, 49.33246777544897, 49.42563329866834, 49.59661410029509]
[88.85459797790945, 86.04299930877899, 59.87779541962817, 41.41757406425071, 26.820806528274137]


TypeError: ignored

In [ ]:
def inc_hebb_train(esn,train_data,test_data,max_epochs,step,train_fun):
  for i in range( esn.N ):
    init_dss = DSSi(esn,test_data,i)
    max_dss = init_dss ;  max_win=np.copy(esn.ress[i].W_in); max_w=np.copy(esn.ress[i].W);

    for epoch in range(max_epochs):
      train_fun(esn,i,train_data,step)

      act_dss = DSSi(esn,test_data,i)

      if max_dss < act_dss : 
        max_dss = act_dss; max_win=np.copy(esn.ress[i].W_in); max_w=np.copy(esn.ress[i].W);
      
    esn.ress[i].W_in = max_win;  esn.ress[i].W = max_w;


In [ ]:
init_dss=[None]*5
final_dss=[None]*5

for g in range(5):
  print(g)
  esn = DeepESN( N=5, Nr=50, Ny=100 , rho=0.9 , r_density=0.8 , i_density=1 )
  
  init_dss[g] = DSS(esn,test_data)
  
  inc_hebb_train(esn, train_data, test_data, 80, [1e-05,-1e-05], inc_train_both)
  
  final_dss[g] = DSS(esn,test_data)

init_dss=np.array(init_dss)
final_dss=np.array(final_dss)

init_means = [None]*5
final_means = [None]*5
ipms = [None]*5

for i in range(5):
  init_means[i] = np.mean(init_dss[:,i])
  final_means[i] = np.mean(final_dss[:,i])
  ipms[i] = np.mean( np.divide( abs(init_dss[:,i]-final_dss[:,i]), init_dss[:,i])*100 )

print(init_means)
print(final_means)
print(ipms)


0
1
2
3
4
[4.680958956401042, 6.438427465003869, 5.265507456858588, 1.9167598915870012, 1.778422099188828]
[5.617013615895074, 13.234992523736912, 19.668814845218247, 18.19072247355044, 14.822107384590229]
[20.63220014864522, 107.31763269228836, 349.25250112678333, 887.9884367744435, 766.1494421309105]


In [ ]:
init_dss=[None]*5
final_dss=[None]*5

for g in range(5):
  print(g)
  esn = DeepESN( N=5, Nr=50, Ny=100 , rho=0.9 , r_density=0.8 , i_density=1 )
  
  init_dss[g] = DSS(esn,test_data)
  
  inc_hebb_train(esn, train_data, test_data, 80, [1e-05,-1e-05], inc_train_both)
  
  final_dss[g] = DSS(esn,test_data)

init_dss=np.array(init_dss)
final_dss=np.array(final_dss)

init_means = [None]*5
final_means = [None]*5
ipms = [None]*5

for i in range(5):
  init_means[i] = np.mean(init_dss[:,i])
  final_means[i] = np.mean(final_dss[:,i])
  ipms[i] = np.mean( np.divide( abs(init_dss[:,i]-final_dss[:,i]), init_dss[:,i])*100 )

print(init_means)
print(final_means)
print(ipms)


0
1
2
3
4
[4.373365093642851, 6.220207424035628, 7.629513228880542, 5.273964215596116, 2.2643528451514934]
[5.391436891520456, 13.470273157829993, 18.051757651017418, 13.605495572722083, 12.260471466937776]
[23.527031667440742, 128.87564708901098, 203.19269447029535, 224.72997443610402, 718.8776694308945]


In [ ]:

init_mcs=[None]*5
final_mcs=[None]*5

init_dss=[None]*5
final_dss=[None]*5

for g in range(5):
  esn = DeepESN( N=5, Nr=50, Ny=100 , rho=0.9 , r_density=0.2 , i_density=0.1 )
  
  for i in range(esn.N):
    esn.train(train_x,train_y,wash_seq,i)
  init_mcs[g] = MC(esn,test_data)
  init_dss[g] = DSS(esn,test_data)
  
  for i in range( esn.N ):
    best_Win = np.copy(esn.ress[i].W_in)
    best_DSS = DSSi(esn,test_data,i)
    
    print('res:', i )
    for epch in range(80):
      inc_train_both(esn,i,train_data,[1e-05,-1e-05])
      
      act_DSS = DSSi(esn,test_data,i)
      if best_DSS < act_DSS:
        best_DSS = act_DSS
        best_Win = np.copy(esn.ress[i].W_in)

    esn.ress[i].W_in = best_Win
  
  for i in range(esn.N):
    esn.train(train_x,train_y,wash_seq,i)
  final_mcs[g] = MC(esn,test_data)
  final_dss[g] = DSS(esn,test_data)

init_mcs=np.array(init_mcs)
final_mcs=np.array(final_mcs)

init_dss=np.array(init_dss)
final_dss=np.array(final_dss)

init_means = [None]*5
final_means = [None]*5
ipms = [None]*5

for i in range(5):
  init_means[i] = np.mean(init_mcs[:,i])
  final_means[i] = np.mean(final_mcs[:,i])
  ipms[i] = np.mean( np.divide( abs(init_mcs[:,i]-final_mcs[:,i]), init_mcs[:,i])*100 )

print('MC')
print(init_means)
print(final_means)
print(ipms)

init_means = [None]*5
final_means = [None]*5
ipms = [None]*5

for i in range(5):
  init_means[i] = np.mean(init_dss[:,i])
  final_means[i] = np.mean(final_dss[:,i])
  ipms[i] = np.mean( np.divide( abs(init_dss[:,i]-final_dss[:,i]), init_dss[:,i])*100 )

print('DSS')
print(init_means)
print(final_means)
print(ipms)



res: 0
res: 1
res: 2
res: 3
res: 4
res: 0
res: 1
res: 2
res: 3
res: 4
res: 0
res: 1
res: 2
res: 3
res: 4
res: 0
res: 1
res: 2
res: 3
res: 4
res: 0
res: 1
res: 2
res: 3
res: 4
MC
[21.02609251197609, 26.646302816239995, 32.09678042441583, 35.060535059155015, 38.684696579299825]
[17.237330496013417, 15.06518901999292, 14.551901898144006, 14.08334245957956, 11.040786880082218]
[17.578410832913743, 43.374236115797586, 54.41528293692206, 59.576609501941824, 71.48435614822534]
DSS
[4.792868400914158, 4.97344971577468, 3.7729248304055374, 2.7670801030577943, 2.1033346350616116]
[3.2965089789684754, 4.82397875311846, 2.956560508186443, 2.0313973806585843, 2.5738978434692044]
[38.95760974037054, 57.9045919342159, 101.1586050332057, 39.14537530869342, 49.71338991346849]


In [ ]:

init_mcs=[None]*5
final_mcs=[None]*5

init_dss=[None]*5
final_dss=[None]*5

for g in range(5):
  esn = DeepESN( N=5, Nr=50, Ny=100 , rho=0.9 , r_density=0.2 , i_density=0.1 )
  
  for i in range(esn.N):
    esn.train(train_x,train_y,wash_seq,i)
  init_mcs[g] = MC(esn,test_data)
  init_dss[g] = DSS(esn,test_data)
  
  inc_hebb_train(esn, train_data, test_data, 80, [1e-05,-1e-05], inc_train_both)
  
  for i in range(esn.N):
    esn.train(train_x,train_y,wash_seq,i)
  final_mcs[g] = MC(esn,test_data)
  final_dss[g] = DSS(esn,test_data)

init_mcs=np.array(init_mcs)
final_mcs=np.array(final_mcs)

init_dss=np.array(init_dss)
final_dss=np.array(final_dss)

init_means = [None]*5
final_means = [None]*5
ipms = [None]*5

for i in range(5):
  init_means[i] = np.mean(init_mcs[:,i])
  final_means[i] = np.mean(final_mcs[:,i])
  ipms[i] = np.mean( np.divide( abs(init_mcs[:,i]-final_mcs[:,i]), init_mcs[:,i])*100 )

print('MC')
print(init_means)
print(final_means)
print(ipms)

init_means = [None]*5
final_means = [None]*5
ipms = [None]*5

for i in range(5):
  init_means[i] = np.mean(init_dss[:,i])
  final_means[i] = np.mean(final_dss[:,i])
  ipms[i] = np.mean( np.divide( abs(init_dss[:,i]-final_dss[:,i]), init_dss[:,i])*100 )

print('DSS')
print(init_means)
print(final_means)
print(ipms)



MC
[20.94553139888568, 28.447241027677386, 32.79988682078225, 37.6230722601094, 41.21720579056422]
[20.55803796361857, 24.798273086700757, 27.746045706570783, 31.136178762518888, 33.70739633447987]
[6.185397899093721, 12.888973989920157, 15.719462986581078, 17.397824954089533, 18.489367548641244]
DSS
[4.10369409467588, 7.146410545961267, 6.019435845027367, 2.4970674914922686, 1.4278569459678883]
[6.029128356446658, 7.3794561284971225, 7.429599174388137, 5.6823208576084046, 5.0807148416384065]
[58.008769314048074, 31.463098008282373, 73.64169754359037, 172.6606833327409, 264.29287332467925]
